In [ ]:
from dotenv import dotenv_values
import helpers
import pandas as pd
import time
import copy
from helpers import musicbrainz,langchainhelpers
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
pd.set_option("display.max_colwidth",1000)

In [ ]:
#load functions of music brainz
all_available_functions = {"searchdetailsrefined":musicbrainz.searchdetailsrefined,
             "getalbumdetails":musicbrainz.getalbumdetails,
             "getartistdetails":musicbrainz.getartistdetails,
             "gettrackdetailsrefined":musicbrainz.gettrackdetailsrefined}

In [ ]:
prompt4 = ChatPromptTemplate.from_messages(
        [
    ("system", "You are an Music Aficionado and know a lot about this field."),
    ("human", "You also have access to a music database, so make calls to the relevant functions if and only if necessary. Remember it costs to call the database and the service isn't cheap."),
    ("human", "Here are some examples to help you out."),
    ("human", "Example 1: Question:  Give me the next song after Golden of the album in which Harry Styles released Golden.\
     Step 1: Decide which entity to query about first: Here it is an track.\
     Step 2: As we have access to a function that returns details about a track, I would call the function asking for details of track Golden. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 3: After receiving the results from the function I would look for the album name.  In this case it would be the album Fine Line.\
     Step 4: As we have access to a function that returns details about an album, I would then call the function asking for details of album Fine Line. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 5: After receiving the results from the function I would then go over all tracks and find the track that is after Golden. In this case it would be the track Watermelon Sugar."),
    ("human", "Example 2: Question: Out of the two, Fine Line and Nobody Is Listening, which is not a Harry Styles music album?\
     Step 1: Decide which entity to query about first: Here it is an album.\
     Step 2: As we have access to a function that returns details about an album, I would call the function asking for details of album Fine Line. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 3: After receiving the results I would look if no results were returned for that album. \
     Step 4: If results were returend for the album, I would repeat steps 2 and 3 with the next album until no results are found. In this case the answer would be the album Nobody Is Listening."),
    ("human", "Example 3: Question: Name the artist who released Love This Giant and was born on 1952.\
     Step 1: Decide which entity to query about: Here it is an album.\
     Step 2: As we have access to a function that returns details about an album, I would call the function asking for details of album Love This Giant.\
     Step 3: After receiving the results from the function I would look for the artists. In this case it would be the artists David Byrne and St. Vincent.\
     Step 4: As we have access to a function that returns details about an artist, I would then call the function asking for details of artist David Byrne.\
     Step 5: After receiving the results I would look if the artist's birth date is 1952.\
     Step 6: If not I would repeat steps 4 and 5 with the next artist until the results are found. In this case it would be the artist David Byrne."),
    ("human", "{input}")
])

# Experimentation

# Only one function available 

In [ ]:
hard_questions_df=pd.read_csv("data/Musicdata/Hard_questions.csv")
hard_questions_df


In [ ]:
results=[]

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__=="searchdetailsrefined"}
available_functions

In [ ]:
for question in tqdm(hard_questions_df[6:7]["Question"]): 
    prompt=copy.deepcopy(prompt4)
    results.append(langchainhelpers.run_llm_chain_more_than_once(prompt,available_functions,question,300,1,5))
    helpers.upload_file("data/Musicdata/intermediate_results.pkl",results)

In [ ]:
helpers.upload_file("data/Musicdata/hard_questions_with_only_function_available_music_brainz_prompt_4.pkl",results)


In [ ]:
results=helpers.download_file("data/Musicdata/hard_questions_with_only_function_available_music_brainz_prompt_4.pkl")

In [ ]:

results=pd.DataFrame(results)
results=pd.merge(left=hard_questions_df,right=results)
results

# 3 subfunctions available 

In [ ]:
hard_questions_df=pd.read_csv("data/Musicdata/Hard_questions.csv")
hard_questions_df

In [ ]:
results=[]

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__!="searchdetailsrefined"}
available_functions

In [ ]:
for question in tqdm(hard_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt4)
    results.append(langchainhelpers.run_llm_chain_more_than_once(prompt,available_functions,question,300,1,5))
    helpers.upload_file("data/Musicdata/intermediate_results.pkl",results)

In [ ]:
results

In [ ]:
helpers.upload_file("data/Musicdata/hard_questions_with_3_subfunctions_available_music_brainz_prompt_4.pkl",results)


In [ ]:
results=helpers.download_file("data/Musicdata/hard_questions_with_3_subfunctions_available_music_brainz_prompt_4.pkl")

In [ ]:

results=pd.DataFrame(results)
results=pd.merge(left=results,right=hard_questions_df)
results

# Only one function available Prompt 5

In [ ]:
hard_questions_df=pd.read_csv("data/Musicdata/Hard_questions.csv")
hard_questions_df

In [ ]:
results=[]
for question in hard_questions_df["Question"]:
    counter=0
    steps={"Question":question}
    steps["steps"]=[]
    prompt = ChatPromptTemplate.from_messages(
       [
    ("system", "You are an Music Aficionado and know a lot about this field."),
    ("human", "You also have access to a music database, so make calls to the relevant functions if and only if necessary. Remember it costs to call the database and the service isn't cheap."),
    ("human", "Here are some examples to help you out."),
    ("human", "Example 1: Question:  Give me the next song after Golden of the album in which Harry Styles released Golden?\
     Step 1: Decide which entity to query about first: Here it is an track.\
     Step 2: As we have access to a function that returns details about a track, I would call the function asking for details of track Golden. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 3: After receiving the results from the function I would look for the album name.  In this case it would be the album Fine Line.\
     Step 4: As we have access to a function that returns details about an album, I would then call the function asking for details of album Fine Line. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 5: After receiving the results from the function I would then go over all tracks and find the track that is after Golden. In this case it would be the track Watermelon Sugar.\
     Final Answer: Watermelon Sugar"),
    ("human", "Example 2: Question: Out of the two, Fine Line and Nobody Is Listening, which is not a Harry Styles's music albums?\
     Step 1: Decide which entity to query about first: Here it is an album.\
     Step 2: As we have access to a function that returns details about an album, I would call the function asking for details of album Fine Line. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 3: After receiving the results I would look if no results were returned for that album. \
     Step 4: If results were returend for the album, I would repeat steps 2 and 3 with the next album until no results are found. In this case the answer would be the album Nobody Is Listening.\
     Final Answer: Nobody Is Listening"),
    ("human", "Example 3: Question: Name the artist who released Love This Giant and was born on 1952?\
     Step 1: Decide which entity to query about: Here it is an album.\
     Step 2: As we have access to a function that returns details about an album, I would call the function asking for details of album Love This Giant.\
     Step 3: After receiving the results from the function I would look for the artists. In this case it would be the artists David Byrne and St. Vincent.\
     Step 4: As we have access to a function that returns details about an artist, I would then call the function asking for details of artist David Byrne.\
     Step 5: After receiving the results I would look if the artist's birth date is 1952.\
     Step 6: If not I would repeat steps 4 and 5 with the next artist until the results are found. In this case it would be the artist David Byrne.\
     Final Answer: David Byrne"),
    ("human", "{input}")
])
    function_llm_chain = create_openai_fn_chain([available_functions["searchdetailsrefined"]], llm, prompt, verbose=True,output_parser=output_parser)
    output=function_llm_chain.run(input=question)
    
    while 'function_call' in output["Message"]:
        counter+=1
        try:
            function_name=output['Parsed_Function_Call']["name"]
            function_arguments=output['Parsed_Function_Call']["arguments"]
        except:
            steps["answer"]=output["Message"]
            break
        
        if function_name in available_functions:
            steps["steps"].append({"Function_name":function_name,"Function_arguments":function_arguments})
            retrieved_results=available_functions[function_name](**function_arguments)
            prompt.extend([AIMessage(content=str(output["Message"]))])
            prompt.extend([FunctionMessage(content=str(retrieved_results),name=function_name)])
            try:
                output=function_llm_chain.run(input=question)
            except:
                steps["answer"]="Wrong output from LLM"
                break
        else:
            steps["steps"].append({"Function_name":function_name,"Function_arguments":function_arguments})
            steps["answer"]="Wrong function called"
            break
        
        if counter==5:
            steps["answer"]="Too many function calls"
            break
        time.sleep(30)
       
    if "answer" not in steps:
        steps["answer"]=output["Message"]
    results.append(steps)
function_llm_chain.llm_kwargs


In [ ]:
results

In [ ]:
helpers.upload_file("data/Musicdata/hard_questions_with_only_function_available_music_brainz_prompt_5.pkl",results)



In [ ]:
results=helpers.download_file("data/Musicdata/hard_questions_with_only_function_available_music_brainz_prompt_5.pkl")



In [ ]:
results=pd.DataFrame(results)
results=pd.merge(left=results,right=hard_questions_df)
results

# 3 subfunctions available Prompt 5 

In [ ]:
hard_questions_df=pd.read_csv("data/Musicdata/Hard_questions.csv")
hard_questions_df

In [ ]:
results=[]

In [ ]:
for question in hard_questions_df[5:10]["Question"]:
    counter=0
    steps={"Question":question}
    steps["steps"]=[]
    prompt = ChatPromptTemplate.from_messages(
       [
    ("system", "You are an Music Aficionado and know a lot about this field."),
    ("human", "You also have access to a music database, so make calls to the relevant functions if and only if necessary. Remember it costs to call the database and the service isn't cheap."),
    ("human", "Here are some examples to help you out."),
    ("human", "Example 1: Question:  Give me the next song after Golden of the album in which Harry Styles released Golden?\
     Step 1: Decide which entity to query about first: Here it is an track.\
     Step 2: As we have access to a function that returns details about a track, I would call the function asking for details of track Golden. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 3: After receiving the results from the function I would look for the album name.  In this case it would be the album Fine Line.\
     Step 4: As we have access to a function that returns details about an album, I would then call the function asking for details of album Fine Line. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 5: After receiving the results from the function I would then go over all tracks and find the track that is after Golden. In this case it would be the track Watermelon Sugar.\
     Final Answer: Watermelon Sugar"),
    ("human", "Example 2: Question: Out of the two, Fine Line and Nobody Is Listening, which is not a Harry Styles's music albums?\
     Step 1: Decide which entity to query about first: Here it is an album.\
     Step 2: As we have access to a function that returns details about an album, I would call the function asking for details of album Fine Line. I would also fill all the other possible parameters as that would give me the most accurate answer. Here it would be the artist name as Harry Styles.\
     Step 3: After receiving the results I would look if no results were returned for that album. \
     Step 4: If results were returend for the album, I would repeat steps 2 and 3 with the next album until no results are found. In this case the answer would be the album Nobody Is Listening.\
     Final Answer: Nobody Is Listening"),
    ("human", "Example 3: Question: Name the artist who released Love This Giant and was born on 1952?\
     Step 1: Decide which entity to query about: Here it is an album.\
     Step 2: As we have access to a function that returns details about an album, I would call the function asking for details of album Love This Giant.\
     Step 3: After receiving the results from the function I would look for the artists. In this case it would be the artists David Byrne and St. Vincent.\
     Step 4: As we have access to a function that returns details about an artist, I would then call the function asking for details of artist David Byrne.\
     Step 5: After receiving the results I would look if the artist's birth date is 1952.\
     Step 6: If not I would repeat steps 4 and 5 with the next artist until the results are found. In this case it would be the artist David Byrne.\
     Final Answer: David Byrne"),
    ("human", "{input}")
])
    function_llm_chain = create_openai_fn_chain([available_functions["gettrackdetailsrefined"],available_functions["getartistdetails"],available_functions["getalbumdetails"]], llm, prompt, verbose=True,output_parser=output_parser)
    output=function_llm_chain.run(input=question)
    
    while 'function_call' in output["Message"]:
        counter+=1
        try:
            function_name=output['Parsed_Function_Call']["name"]
            function_arguments=output['Parsed_Function_Call']["arguments"]
        except:
            steps["answer"]=output["Message"]
            break
        if function_name in available_functions:
            steps["steps"].append({"Function_name":function_name,"Function_arguments":function_arguments})
            retrieved_results=available_functions[function_name](**function_arguments)
            prompt.extend([AIMessage(content=str(output["Message"]))])
            prompt.extend([FunctionMessage(content=str(retrieved_results),name=function_name)])
            output=function_llm_chain.run(input=question)
        else:
            steps["steps"].append({"Function_name":function_name,"Function_arguments":function_arguments})
            steps["answer"]="Wrong function called"
            break
        
        if counter==5:
            steps["answer"]="Too many function calls"
            break
        time.sleep(30)
       
    if "answer" not in steps:
        steps["answer"]=output["Message"]
    results.append(steps)
    
function_llm_chain.llm_kwargs


In [ ]:
results

In [ ]:
helpers.upload_file("data/Musicdata/hard_questions_with_3_subfunctions_available_music_brainz_prompt_5.pkl",results)

In [ ]:
results=helpers.download_file("data/Musicdata/hard_questions_with_3_subfunctions_available_music_brainz_prompt_5.pkl")

In [ ]:


results=pd.DataFrame(results)
results=pd.merge(left=results,right=hard_questions_df)
results
